In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch

In [3]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer

# Load the dataset
csv_path = "/content/Source_Code_Plagiarism_Processed.csv"
df = pd.read_csv(csv_path)

# Map labels to numerical classes
label_mapping = {"Non-Plagiarized": 0, "L1": 1, "L2": 2, "L3": 3, "L4": 4, "L5": 5, "L6": 6}
df['label'] = df['PlagiarismLevel'].map(label_mapping)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
test_dataset = dataset['test']

# Load CodeT5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")  # Change this if using CodeBERT or another model

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples['Code'],
        examples['OriginalCode'],
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns after tokenization
train_dataset = train_dataset.remove_columns(["Code", "OriginalCode", "PlagiarismLevel"])
test_dataset = test_dataset.remove_columns(["Code", "OriginalCode", "PlagiarismLevel"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Map:   0%|          | 0/368 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(
    "Salesforce/codet5-base",
    num_labels=len(label_mapping)
)

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir="./codet5_multiclass",
    run_name="CodeT5_MultiClass",  # Unique run name
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

<ipython-input-16-4593ec5e8e8a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [18]:
!wandb login

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.769400,1.837451
2,1.687700,1.792571
3,1.509300,1.762205
4,1.282500,1.600220
5,1.284500,1.443598
6,1.389300,1.581506
7,1.127500,1.496282
8,1.121400,1.425999


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=368, training_loss=1.4350159038668093, metrics={'train_runtime': 653.5917, 'train_samples_per_second': 4.504, 'train_steps_per_second': 0.563, 'total_flos': 1798162008637440.0, 'train_loss': 1.4350159038668093, 'epoch': 8.0})

In [20]:
output_dir = "./codet5_finetuned_multiclass"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./codet5_finetuned_multiclass/tokenizer_config.json',
 './codet5_finetuned_multiclass/special_tokens_map.json',
 './codet5_finetuned_multiclass/vocab.json',
 './codet5_finetuned_multiclass/merges.txt',
 './codet5_finetuned_multiclass/added_tokens.json',
 './codet5_finetuned_multiclass/tokenizer.json')

In [21]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 1.425998568534851, 'eval_runtime': 4.5982, 'eval_samples_per_second': 20.008, 'eval_steps_per_second': 2.61, 'epoch': 8.0}


In [31]:
from transformers import pipeline

# Load the fine-tuned model
codet5_pipeline = pipeline(
    "text-classification",
    model=output_dir,
    tokenizer=output_dir
)

# Example input
example_code = """
public class T1 {
	public static void main(String[] args) {
		System.out.println("Welcome to Java");
		System.out.println("Welcome to Java");
		System.out.println("Welcome to Java");
		System.out.println("Welcome to Java");
		System.out.println("Welcome to Java");
	}

}
"""
example_original = """
public class Main {
    public static void main(String[] args) {
        cetak();
    }
    public static void cetak(){
        for (int i = 0; i < 5; i++) {
            System.out.println("Welcome to Java");

        }
    }
}

"""

# Define the multi-class prompt
prompt = f"""
Task: Classify the level of plagiarism between the two code snippets.
Focus: Compare the coding logic, structure, and functionality. Ignore minor differences like variable names or formatting.
Output: Provide one of the following labels:
- "Non-Plagiarized": The second code is independently written.
- "L1": The second code is almost identical to the original with minor changes like renaming variables or reformatting.
- "L2": The second code has some structural changes while preserving the logic.
- "L3": Moderate changes to logic or structure with significant similarities to the original.
- "L4": Heavy modifications with some recognizable similarity to the original.
- "L5": Significant reorganization or obfuscation, making the similarity less obvious.
- "L6": Highly transformed but still derived from the original.

Original Code:
{example_original}

Submission Code:
{example_code}
"""


# Get predictions
result = codet5_pipeline(prompt)
print("Prediction:", result)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prediction: [{'label': 'LABEL_5', 'score': 0.36001163721084595}]


In [32]:
!pip install huggingface_hub

In [33]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `CodeT5` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [34]:
output_dir = "./codet5_multiclass"  # Change the path as needed
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./codet5_multiclass/tokenizer_config.json',
 './codet5_multiclass/special_tokens_map.json',
 './codet5_multiclass/vocab.json',
 './codet5_multiclass/merges.txt',
 './codet5_multiclass/added_tokens.json',
 './codet5_multiclass/tokenizer.json')

In [35]:
from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub("YoussefHassan/codet5-multiclass-plagiarism-detector")
tokenizer.push_to_hub("YoussefHassan/codet5-multiclass-plagiarism-detector")

print("Model uploaded successfully.")


model.safetensors:   0%|          | 0.00/894M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model uploaded successfully.
